# Albert School Facial Recognition System

The Albert School currently has a badge entry system to access the school grounds.
We propose to the school to continue its IT momentum by replacing this system with a facial recognition system.
Here is the prototype:

## The system

So we have the following data base

In [31]:
pip install facenet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd

In [34]:
df_albert = pd.read_csv('db_student.csv', sep=';')

df_albert

,student_id,name,first_name,birth_date,age,path_picture
0,1,Hoda,Dades,12/04/1999,25,hoda_cropped.png
1,2,Yasmine,Ferdjani,25/08/2000,24,yasmine_cropped.png


In this data base we have two students. Each line means an entry in the area of the school.
So Hoda and Yasmine cames only once in the school. 

In [35]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install watchdog

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import pandas as pd
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch

# Initialisation de MTCNN pour la détection faciale et InceptionResnetV1 pour les embeddings
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def compare_images(img1_path, img2_path, threshold=0.8):
    """
    Compare deux images en calculant la distance entre leurs embeddings faciaux.
    """
    
    img1 = Image.open(img1_path)
    img2 = Image.open(img2_path)
    
    # Détecter et recadrer les visages dans les deux images
    img1_cropped = mtcnn(img1)
    img2_cropped = mtcnn(img2)
    
    if img1_cropped is None or img2_cropped is None:
        print("Erreur : visage non détecté dans l'une des images.")
        return False, None
    
    # Calculer les embeddings des deux images
    img1_embedding = resnet(img1_cropped.unsqueeze(0))
    img2_embedding = resnet(img2_cropped.unsqueeze(0))
    
    # Calculer la distance entre les deux embeddings (distance Euclidienne)
    distance = torch.dist(img1_embedding, img2_embedding).item()
    
    # Comparer la distance avec le seuil
    return distance < threshold, distance

def compare_folder_to_df(folder_path, df, threshold=0.8):
    """
    Compare chaque photo dans un dossier avec les photos de la DataFrame.
    
    Args:
    - folder_path: Chemin vers le dossier contenant les photos à comparer.
    - df: DataFrame contenant les informations des étudiants avec les chemins des photos.
    - threshold: Seuil pour déterminer si les visages correspondent.
    
    Returns:
    - Une liste de résultats de correspondance pour chaque photo dans le dossier.
    """
    
    results = []
    entry_in_campus = []
    valid_extensions = ('.jpg', '.jpeg', '.png')  # Extensions de fichiers image valides
    
    # Parcourir toutes les images dans le dossier
    for img_file in os.listdir(folder_path):
        # Ignorer les fichiers qui ne sont pas des images
        if not img_file.lower().endswith(valid_extensions):
            continue
        
        img_path = os.path.join(folder_path, img_file)
        
        # Comparer avec chaque image dans la DataFrame
        for idx, row in df.iterrows():
            ref_img_path = row['path_picture']
            
            # Comparer l'image du dossier avec celle de la DataFrame
            match, distance = compare_images(img_path, ref_img_path, threshold)
            
            # Ajouter le résultat dans la liste des résultats
            results.append({
                'folder_image': img_file,
                'student_id': row['student_id'],
                'student_name': row['name'],
                'match': match,
                'distance': distance
            })
            
            # Si une correspondance est trouvée, ajouter les informations de l'étudiant
            if match:
                entry_in_campus.append({
                    'student_id': row['student_id'],
                    'name': row['name'],
                    'first_name': row['first_name'],
                    'birth_date': row['birth_date'],
                    'age': row['age'],
                    'path_picture': row['path_picture']
                })
    
    # Retourner les résultats des correspondances et la liste des étudiants correspondants
    results_df = pd.DataFrame(results)
    entry_in_campus_df = pd.DataFrame(entry_in_campus)
    
    return results_df, entry_in_campus_df

# Exemple d'utilisation
folder_path = 'pictures_from_entry' 

# Comparaison des photos du dossier avec la base de données
result_df, entry_in_campus_df = compare_folder_to_df(folder_path, df_albert)

# Afficher les résultats
print("Résultats de la correspondance :")
print(result_df)

print("\nÉtudiants correspondants :")
print(entry_in_campus_df)

Résultats de la correspondance :
                 folder_image  student_id student_name  match  distance
0              yasmine_2.jpeg           1         Hoda  False  1.315818
1              yasmine_2.jpeg           2      Yasmine   True  0.467260
2            hoda_cropped.png           1         Hoda   True  0.000000
3            hoda_cropped.png           2      Yasmine  False  1.277669
4   yasmine_cropped copie.png           1         Hoda  False  1.277669
5   yasmine_cropped copie.png           2      Yasmine   True  0.000000
6         yasmine_cropped.png           1         Hoda  False  1.277669
7         yasmine_cropped.png           2      Yasmine   True  0.000000
8              moustapha.jpeg           1         Hoda  False  1.219394
9              moustapha.jpeg           2      Yasmine  False  1.461664
10   hoda_cropped copie 2.png           1         Hoda   True  0.000000
11   hoda_cropped copie 2.png           2      Yasmine  False  1.277669
12     hoda_cropped copie.png  

In [23]:
entry_in_campus_df.to_csv('entry_in_campus.csv', index=False)

In [16]:
#pip install flask


In [ ]:
import asyncio
from IPython.display import display
from ipywidgets import Button, Output

output = Output()

# Cette fonction sera appelée lorsqu'un signal WebSocket est reçu
async def execute_cell_on_trigger():
    with output:
        print("Photo taken! Executing the Python code...")

import os
import pandas as pd
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch

# Initialisation de MTCNN pour la détection faciale et InceptionResnetV1 pour les embeddings
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

def compare_images(img1_path, img2_path, threshold=0.8):
    """
    Compare deux images en calculant la distance entre leurs embeddings faciaux.
    """
    
    img1 = Image.open(img1_path)
    img2 = Image.open(img2_path)
    
    # Détecter et recadrer les visages dans les deux images
    img1_cropped = mtcnn(img1)
    img2_cropped = mtcnn(img2)
    
    if img1_cropped is None or img2_cropped is None:
        print("Erreur : visage non détecté dans l'une des images.")
        return False, None
    
    # Calculer les embeddings des deux images
    img1_embedding = resnet(img1_cropped.unsqueeze(0))
    img2_embedding = resnet(img2_cropped.unsqueeze(0))
    
    # Calculer la distance entre les deux embeddings (distance Euclidienne)
    distance = torch.dist(img1_embedding, img2_embedding).item()
    
    # Comparer la distance avec le seuil
    return distance < threshold, distance

def compare_folder_to_df(folder_path, df, threshold=0.8):
    """
    Compare chaque photo dans un dossier avec les photos de la DataFrame.
    
    Args:
    - folder_path: Chemin vers le dossier contenant les photos à comparer.
    - df: DataFrame contenant les informations des étudiants avec les chemins des photos.
    - threshold: Seuil pour déterminer si les visages correspondent.
    
    Returns:
    - Une liste de résultats de correspondance pour chaque photo dans le dossier.
    """
    
    results = []
    entry_in_campus = []
    valid_extensions = ('.jpg', '.jpeg', '.png')  # Extensions de fichiers image valides
    
    # Parcourir toutes les images dans le dossier
    for img_file in os.listdir(folder_path):
        # Ignorer les fichiers qui ne sont pas des images
        if not img_file.lower().endswith(valid_extensions):
            continue
        
        img_path = os.path.join(folder_path, img_file)
        
        # Comparer avec chaque image dans la DataFrame
        for idx, row in df.iterrows():
            ref_img_path = row['path_picture']
            
            # Comparer l'image du dossier avec celle de la DataFrame
            match, distance = compare_images(img_path, ref_img_path, threshold)
            
            # Ajouter le résultat dans la liste des résultats
            results.append({
                'folder_image': img_file,
                'student_id': row['student_id'],
                'student_name': row['name'],
                'match': match,
                'distance': distance
            })
            
            # Si une correspondance est trouvée, ajouter les informations de l'étudiant
            if match:
                entry_in_campus.append({
                    'student_id': row['student_id'],
                    'name': row['name'],
                    'first_name': row['first_name'],
                    'birth_date': row['birth_date'],
                    'age': row['age'],
                    'path_picture': row['path_picture']
                })
    
    # Retourner les résultats des correspondances et la liste des étudiants correspondants
    results_df = pd.DataFrame(results)
    entry_in_campus_df = pd.DataFrame(entry_in_campus)
    
    return results_df, entry_in_campus_df

# Exemple d'utilisation
folder_path = 'pictures_from_entry' 

# Comparaison des photos du dossier avec la base de données
result_df, entry_in_campus_df = compare_folder_to_df(folder_path, df_albert)

# Afficher les résultats
print("Résultats de la correspondance :")
print(result_df)

print("\nÉtudiants correspondants :")
print(entry_in_campus_df)

display(output)

# Code pour surveiller les WebSocket (le WebSocket doit être configuré pour communiquer avec le HTML)
import websockets

async def listen():
    uri = "ws://localhost:8888"  # L'URL du WebSocket
    async with websockets.connect(uri) as websocket:
        while True:
            message = await websocket.recv()
            if message == '{"action": "take_photo_trigger"}':
                await execute_cell_on_trigger()

# Lancer la boucle WebSocket
asyncio.get_event_loop().run_until_complete(listen())